In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2


In [ ]:
df_unclean = pd.read_csv('NPRI_Data_2022_unclean (1).csv')
df_review = pd.read_csv('NPRI Data for QC.xlsx - NPRI Data for review (3).csv')

<ipython-input-356-86ca4702da84>:2: DtypeWarning:

Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.



In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Making Copy of orginal Data set

In [ ]:
df_unc = df_unclean.copy()
df_rev = df_review.copy()

In [ ]:
df_unc.head()

# EDA on Unclean Dataset

In [ ]:
df_unc.describe()

In [ ]:
df_unc.shape

(14818, 30)

In [ ]:
df_unc.isnull().sum()

In [ ]:
df_unc.dtypes

# Aligning

In this df_unc, the quantity of columens recorded in Tonnes,KG, grams, and g_teq_et. So we have to convert all these into a single unit. For this dataset, We convert all the units in KG.

In [ ]:
list_columns = ["CurrentYear","YearMinus1","YearMinus2","YearMinus3","YearMinus4"]
for i in list_columns:
    # Convert tonnes to grams
    tonnes_conv = df_unc.loc[df_unc["Units"] == "tonnes", i] * 1000
    df_unc.loc[df_unc["Units"] == "tonnes", i] = tonnes_conv

    # Convert grams to kilograms
    grams_conv = df_unc.loc[df_unc["Units"] == "grams", i] / 1000
    df_unc.loc[df_unc["Units"] == "grams", i] = grams_conv

    # Convert g_teq_et to kilograms
    g_teq_et_conv = df_unc.loc[df_unc["Units"] == "g_teq_et", i] / 1000
    df_unc.loc[df_unc["Units"] == "g_teq_et", i] = g_teq_et_conv

# Handeling missing vlaues

We have empty values in the columnes Sub-Category,YearMinus1,               
YearMinus2,               
YearMinus3,               
YearMinus4,               
PercentChange,
FourYearAvg,
NumStandardDeviation,
SubstanceComments.

Out of the Sub-Category and SubstanceComments are catogrical featurs. Rest of the data is numerical.

In [ ]:
df_unc.head(4)

,IssueID,ReportID,Priority,IssueType,ReportYear,NpriID,CompanyName,FacilityName,NAICS,Category,Sub-Category,Cas,SubstanceName,Units,CurrentYear,YearMinus1,YearMinus2,YearMinus3,YearMinus4,PercentChange,DifferenceChange,FourYearAvg,NAICSSubstanceAvg,NumStandardDeviation,StandardDeviation,NAICSSubstanceCount,NAICSSubstanceTotal,OverallSubstanceTotal,NumOfEmployees,SubstanceComments
0,57733,335210,6,Trend,2022,11017,City of Kawartha Lakes,Lindsay WPCP,221320,Water,Direct Discharges,NA - 17,Nitrate ion in solution at pH >= 6.0,tonnes,39740.0,22640.0,18830.0,35600.0,19600.0,0.755300,17.1000,24.167500,726.020055,-0.355612,1929.857872,101,73328.025600,81561.568318,5,Nature of activities results in annual variati...
1,57740,335213,6,Trend,2022,10979,City of Owen Sound,Owen Sound WPCP,221320,DisposalsTransfers,Sum of Disposals and Transfers,NA - 16,Ammonia (total),tonnes,20964.9,10984.6,19852.2,19820.0,13039.2,0.908572,9.9803,15.924000,106.904767,-0.226340,379.694303,105,11225.000530,33430.126126,6,Nature of treating wastewater changes daily. T...
2,57745,335213,6,Trend,2022,10979,City of Owen Sound,Owen Sound WPCP,221320,Water,Spills,NA - 16,Ammonia (total),tonnes,57.0,8798.8,875.0,333.0,1045.0,-0.993522,-8.7418,2.762950,264.082095,-0.312018,846.184135,166,43837.627693,46252.286544,6,Quantities change year to year due to the natu...
3,57746,335213,6,Trend,2022,10979,City of Owen Sound,Owen Sound WPCP,221320,Water,Spills,NA - 22,Phosphorus (total),tonnes,23.0,3941.7,598.0,183.0,498.0,-0.994165,-3.9187,1.305175,28.126980,-0.377584,74.431036,149,4190.920093,5418.433552,6,Nature of treating wastewater changes daily. t...


In [ ]:
# Scatter Plot
import plotly.express as px

fig = px.scatter(df_unc, x="YearMinus1", y="YearMinus2", )
fig.show()


In [ ]:
# Heat Map

fig = px.imshow(df_unc[['CurrentYear', 'YearMinus1', 'YearMinus2', 'YearMinus3', 'YearMinus4', 'PercentChange', ]].corr(),
              text_auto=True)
fig.show()


To fill these ,'YearMinus1', 'YearMinus2', 'YearMinus3', 'YearMinus4',"PercentChange","FourYearAvg","NumStandardDeviation" columns we are using means. The reason, all the data we have in this unclean dataset is 2022. Therefor, we use LOCF AND NOCB method to fill the missing value.

In the case of handeling missng vlaus in categorical featueas as
SubstanceComments        9963
Sub-Category             3277
We can use Mode to fill the values. But by doing this, the dataset might be bias. However considering the fact that, the amount of missing vlause is more then 10% of whole dataset and the relavnce to target columns for this two feature much less. Therefor, instead of imputing it,we can remove these columns.

In [ ]:
df_unc.isnull().sum()

IssueID                     0
ReportID                    0
Priority                    0
IssueType                   0
ReportYear                  0
NpriID                      0
CompanyName                 0
FacilityName                0
NAICS                       0
Category                    0
Sub-Category             3277
Cas                         0
SubstanceName               0
Units                       0
CurrentYear                 0
YearMinus1                108
YearMinus2               2664
YearMinus3               4156
YearMinus4               5181
PercentChange             110
DifferenceChange            0
FourYearAvg                77
NAICSSubstanceAvg           0
NumStandardDeviation      781
StandardDeviation           0
NAICSSubstanceCount         0
NAICSSubstanceTotal         0
OverallSubstanceTotal       0
NumOfEmployees              0
SubstanceComments        9963
dtype: int64

In [ ]:
list_colums = ['YearMinus1',"PercentChange"]
for i in list_colums:
  df_unc[i] = df_unc[i].fillna(method='ffill')
  df_unc[i] = df_unc[i].fillna(method='bfill')



In [ ]:
list_colums = ['YearMinus2', 'YearMinus3', 'YearMinus4',"FourYearAvg","NumStandardDeviation"]
for i in list_colums:
  df_unc[i] = df_unc[i].interpolate(method='linear')


**Finding Outlier using univariate way**

In [ ]:
def remove_outliers(df, columns):
    """Remove outliers from specified columns using box plot statistics."""
    df_clean = df.copy()
    for column in columns:
        Q1 = df_clean[column].quantile(0.25)
        Q3 = df_clean[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_clean = df_clean[(df_clean[column] >= lower_bound) & (df_clean[column] <= upper_bound)]
    return df_clean

# Specify the columns for outlier removal
columns_to_remove_outliers = ['YearMinus1', 'YearMinus2', 'YearMinus3', 'YearMinus4',
                              'PercentChange', 'DifferenceChange', 'FourYearAvg',
                              'NAICSSubstanceAvg', 'NumStandardDeviation', 'StandardDeviation',
                              'NAICSSubstanceCount', 'NAICSSubstanceTotal', 'OverallSubstanceTotal',
                              'NumOfEmployees']

# Assuming df_unc is your DataFrame containing the data
df_clean = remove_outliers(df_unc, columns_to_remove_outliers)

**Finding Outlier using multivariate way**  

In [ ]:

def mahalanobis_distance(x, data):
    """Compute Mahalanobis Distance for a data point x from the data."""
    cov = np.cov(data, rowvar=False)
    cov_reg = cov + np.eye(cov.shape[0]) * 1e-6  # Regularization term
    cov_inv = np.linalg.inv(cov_reg)
    mean = np.mean(data, axis=0)
    diff = x - mean
    return np.sqrt(np.dot(np.dot(diff, cov_inv), diff.T))

def detect_outliers(df):
    """Detect outliers in a DataFrame using Mahalanobis Distance."""
    # Drop non-numeric columns if any
    numeric_df = df.select_dtypes(include=[np.number])
    data = numeric_df.values
    outliers = []
    for i in range(len(data)):
        x = data[i]
        m_dist = mahalanobis_distance(x, data)
        # Set the significance level for outlier detection
        threshold = chi2.ppf(0.95, df=data.shape[1])
        if m_dist > threshold:
            outliers.append(i)
    return outliers

# Assuming df_unc is your DataFrame containing the data
outliers_indices = detect_outliers(df_unc)


In [ ]:
df_unc.drop(outliers_indices, inplace=True)


Removing Any duplicate vlaue from the data using duplicate funcion

In [ ]:
df_unc_cleaned = df_unc.drop_duplicates()

# EDA on Review Dataset

In [ ]:
df_rev.isnull().sum()

NpriID                                                          0
Company name                                                   10
FacilityName                                                  165
NPRI_Report_ReportYear                                          0
Quantity                                                      954
Cas                                                             0
Detail_NPRI_SubstanceInfo_NameEn                                0
Group                                                           0
Category                                                        0
Latitude                                                        0
Longitude                                                       0
ProvinceEn                                                      0
Detail_UnitType_NameEn                                          0
NAICSPrimary                                                    0
NAICSTitleEn                                                    0
Substance 

In [ ]:
df_rev.shape

(537550, 18)

In [ ]:
df_rev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537550 entries, 0 to 537549
Data columns (total 18 columns):
 #   Column                                                   Non-Null Count   Dtype  
---  ------                                                   --------------   -----  
 0   NpriID                                                   537550 non-null  int64  
 1   Company name                                             537540 non-null  object 
 2   FacilityName                                             537385 non-null  object 
 3   NPRI_Report_ReportYear                                   537550 non-null  int64  
 4   Quantity                                                 536596 non-null  float64
 5   Cas                                                      537550 non-null  object 
 6   Detail_NPRI_SubstanceInfo_NameEn                         537550 non-null  object 
 7   Group                                                    537550 non-null  object 
 8   Category      

In [ ]:
df_rev.isnull().sum()

NpriID                                                          0
Company name                                                   10
FacilityName                                                  165
NPRI_Report_ReportYear                                          0
Quantity                                                      954
Cas                                                             0
Detail_NPRI_SubstanceInfo_NameEn                                0
Group                                                           0
Category                                                        0
Latitude                                                        0
Longitude                                                       0
ProvinceEn                                                      0
Detail_UnitType_NameEn                                          0
NAICSPrimary                                                    0
NAICSTitleEn                                                    0
Substance 

**Standerizing the unit columns into Kilo Grame**

In [ ]:
list_columns = ["Quantity"]
for i in list_columns:
  tonnes_conv =df_rev.loc[df_rev["Detail_UnitType_NameEn"] =="tonnes",i]*1000
  df_rev.loc[df_rev["Detail_UnitType_NameEn"] =="tonnes",i] = tonnes_conv

  grams_conv=df_rev.loc[df_rev["Detail_UnitType_NameEn"] =="grams",i]/1000
  df_rev.loc[df_rev["Detail_UnitType_NameEn"] =="grams",i] = grams_conv

  g_teq_et_conv =df_rev.loc[df_rev["Detail_UnitType_NameEn"] =="g_teq_et",i]/1000
  df_rev.loc[df_rev["Detail_UnitType_NameEn"] =="g_teq_et",i] = g_teq_et_conv

**Handeling Missing Value**

In this dataset we have reported year from 2014 to 2022. Therefor, to fill featuers such as Substance comments by category for 2022,                    
Substance comments for disposals and all media for 2022,
Quantity,
Company name,
FacilityName  
We can't use mean, meadian or mode. In this case we can use NOCB and LOCF or interpolation to fill the values.
********************************************************************************
featuers such as Substance comments by category for 2022,                    
Substance comments for disposals and all media for 2022,

We have 451554 and 537479 missing values in this two columns. Becuase the missing values more 80% of total data doing imputaion might indrouce bais in our dataset. By considering the relavece to the target columns we can remove these two columns.

We will drop these two columns in phase of featuer selection

**Data Leakage**
Before we filling the missing valuse, we have split the dataset into two. One dataset have data for the year 2022 which will our test data and other one have all the data from 2014 to 2021.

In [ ]:
df_rev_2022 = df_rev[df_rev['NPRI_Report_ReportYear'] == 2022].copy()
df_rev_2014_21 = df_rev[df_rev['NPRI_Report_ReportYear'] != 2022].copy()

In [ ]:
list_columns = ["Company name", "FacilityName"]
df_rev_2014_21 = df_rev_2014_21.dropna(subset=list_columns)
df_rev_2022 = df_rev_2022.dropna(subset=list_columns)


In [ ]:
df_rev_2022.dropna(subset=["Quantity"], inplace=True)
df_rev_2014_21.dropna(subset=["Quantity"], inplace=True)

In [ ]:
temp_1 = df_rev_2022["Quantity"]
temp_2 = df_rev_2014_21["Quantity"]

In [ ]:
df_rev_2022.drop(columns=["Quantity"], inplace=True)
df_rev_2014_21.drop(columns=["Quantity"], inplace=True)

**Handling  outliers**. AS per client, we have to find the outliers by sector. To do that we have to splite the dataset by sectors. For the split we use NAICSTitleEn columns have different sectors.

In [ ]:
df_rev_2022["NAICSTitleEn"].unique()


In [ ]:
df_rev_2014_21["NAICSTitleEn"].unique()

In [ ]:
diff_sec = df_rev_2022["NAICSTitleEn"].unique()
for x in diff_sec:
    temp_01 = df_rev_2022[df_rev_2022["NAICSTitleEn"] == x]
    def remove_outliers(df, columns):
      """Remove outliers from specified columns using box plot statistics."""
      df_clean = df.copy()
      for column in columns:
          Q1 = df_clean[column].quantile(0.25)
          Q3 = df_clean[column].quantile(0.75)
          IQR = Q3 - Q1
          lower_bound = Q1 - 1.5 * IQR
          upper_bound = Q3 + 1.5 * IQR
          df_clean = df_clean[(df_clean[column] >= lower_bound) & (df_clean[column] <= upper_bound)]
      return df_clean

    # Specify the columns for outlier removal
    columns_to_remove_outliers = ['Quantity']
    df_clean = remove_outliers(temp_01, columns_to_remove_outliers)

In [ ]:
diff_sec = df_rev_2014_21["NAICSTitleEn"].unique()
for x in diff_sec:
    temp_01 = df_rev_2014_21[df_rev_2014_21["NAICSTitleEn"] == x]
    def remove_outliers(df, columns):
      """Remove outliers from specified columns using box plot statistics."""
      df_clean = df.copy()
      for column in columns:
          Q1 = df_clean[column].quantile(0.25)
          Q3 = df_clean[column].quantile(0.75)
          IQR = Q3 - Q1
          lower_bound = Q1 - 1.5 * IQR
          upper_bound = Q3 + 1.5 * IQR
          df_clean = df_clean[(df_clean[column] >= lower_bound) & (df_clean[column] <= upper_bound)]
      return df_clean

    # Specify the columns for outlier removal
    columns_to_remove_outliers = ['Quantity']

    # Assuming df_unc is your DataFrame containing the data
    df_clean = remove_outliers(temp_01, columns_to_remove_outliers)

**Finding Outlier using multivariate way for 2022 dataset**

In [ ]:
from scipy.stats import chi2

def mahalanobis_distance(x, data):
    """Compute Mahalanobis Distance for a data point x from the data."""
    cov = np.cov(data, rowvar=False)
    cov_reg = cov + np.eye(cov.shape[0]) * 1e-6  # Regularization term
    cov_inv = np.linalg.inv(cov_reg)
    mean = np.mean(data, axis=0)
    diff = x - mean
    return np.sqrt(np.dot(np.dot(diff, cov_inv), diff.T))
def detect_outliers(df):
    """Detect outliers in a DataFrame using Mahalanobis Distance."""
    # Drop non-numeric columns if any
    numeric_df = df.select_dtypes(include=[np.number])
    data = numeric_df.values
    outliers = []
    if len(data) < 2:
        print("Skipping outlier detection: Insufficient data points.")
        return outliers
    for i in range(len(data)):
        x = data[i]
        m_dist = mahalanobis_distance(x, data)
        # Set the significance level for outlier detection
        threshold = chi2.ppf(0.95, df=data.shape[1])
        if m_dist > threshold:
            outliers.append(i)
        print(f"Processed {i+1}/{len(data)} data points")
    return outliers

In [ ]:
outliers_indices = detect_outliers(df_rev_2014_21)
df_rev_2014_21 = df_rev_2014_21.drop(outliers_indices)

In [ ]:
outliers_indices = detect_outliers(df_rev_2022)
df_rev_2022= df_rev_2022.drop(outliers_indices)

In [ ]:
df_rev_2022 = pd.concat([temp_1, df_rev_2022], axis=1) # add the removed quantity columns
df_rev_2014_21 = pd.concat([temp_2, df_rev_2014_21], axis=1)

In [ ]:
df_rev_2022.dtypes

In [ ]:
df_rev_2014_21.to_csv('out_lier_dataset.csv', index=False)

Checking for duplicat row. And, if its have any duplicate row the code will remove it.


In [ ]:
df_rev_2022_cleaned = df_rev_2022.drop_duplicates()
df_rev_2014_21_cleaned = df_rev_2014_21.drop_duplicates()


In [ ]:
df_rev_2022.isna().sum()

**Mergeing the dataset**

**After merging we having the missing value so, we will tarin data with data having missing value imputed as well as missing values removed. However, in this example i am imputing the missing value using ffil and bill**

In [ ]:
merg_02 = df_unc[["YearMinus1","YearMinus2","NpriID","Cas"]]

In [ ]:
target_df = pd.merge(merg_02, df_rev_2022, on=['NpriID', 'Cas'], how='inner') # merger yearminus1 and yearminus2 from unclean dataset to merged df which have the data for 2022.

In [ ]:
target_df.isna().sum()

In [ ]:
target_df.dtypes

In [ ]:
target_df = target_df.fillna(method='ffill').fillna(method='bfill')


In [ ]:
tttemp= target_df.copy()


In [ ]:
target_df = tttemp.copy()

In [ ]:
target_df.shape

(30324, 20)

# Encoding,Normalization

**Encoding the dataset**

In [ ]:
list_catg = ["NAICSTitleEn","ProvinceEn","Category","Group","Detail_NPRI_SubstanceInfo_NameEn","FacilityName","Company name","Cas"]

In [ ]:
for catg in list_catg:
    try:
        mean_target = target_df.groupby(catg)['Quantity'].mean()
        target_df[catg + '_mean_encoded'] = target_df[catg].map(mean_target)
    except KeyError as e:
        print(f"KeyError: {e} not found in DataFrame.")

In [ ]:
target_df.head(100)

In [ ]:
target_df.drop(columns=['NAICSTitleEn','ProvinceEn','Category','Group','Detail_NPRI_SubstanceInfo_NameEn','FacilityName','Company name','Cas'], inplace=True)

In [ ]:
list_col = target_df.columns

**Normalization**

In [ ]:


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
target_df[list_col] = scaler.fit_transform(target_df[list_col])

In [ ]:
target_df.isna().sum()

**Feature Selection based on Pearson value and correlation_matrix. After i used these two methond. We will drop 'FacilityName_mean_encoded', 'Category_mean_encoded',
       'Detail_NPRI_SubstanceInfo_NameEn_mean_encoded',
       'FacilityName_mean_encoded', 'Cas_mean_encoded' **

In [ ]:

correlation_matrix = target_df.corr().abs()


target_corr = correlation_matrix['Quantity']


selected_features = target_corr[target_corr > 0.2].index.tolist()


selected_features.remove('Quantity')


print("Selected features based on Pearson correlation coefficient:")
print(selected_features)

Selected features based on Pearson correlation coefficient:
['FacilityName_mean_encoded']


In [ ]:
X_with_const = sm.add_constant(target_df.drop(columns=['Quantity']))

model = sm.OLS(target_df['Quantity'], X_with_const).fit()

print(model.summary())

p_values = model.pvalues

selected_features = target_df.columns[1:][p_values[1:] < 0.05]

print("Selected Features based on p-value less than 0.05:")
print(selected_features)

                            OLS Regression Results                            
Dep. Variable:               Quantity   R-squared:                       0.071
Model:                            OLS   Adj. R-squared:                  0.071
Method:                 Least Squares   F-statistic:                     194.0
Date:                Mon, 08 Apr 2024   Prob (F-statistic):               0.00
Time:                        18:53:43   Log-Likelihood:             1.0679e+05
No. Observations:               30324   AIC:                        -2.136e+05
Df Residuals:                   30311   BIC:                        -2.135e+05
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                                                    coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------

In [ ]:
df_rev_2014_21.shape

(471384, 18)

# ML_EDA

** feature engineering. Constructing new columns minus one and two based on quantity. To do this action, we need to go by each row then find the previous year same company name and the same type of substance. Unfortunately, we have a big data therefor, to do this action It will take hours. Therefore, based of our ML problem we made new df which have the data for oil and sands industry. In this df we only 7000 data **

In [ ]:
list_name = ["In-situ oil sands extraction", "Mined oil sands extraction"]
for i in list_name:
    target_df_1 = target_df[target_df["NAICSTitleEn"] == i].copy()


In [ ]:
target_df_1.shape

(1303, 20)

In [ ]:
list_name = ["In-situ oil sands extraction","Mined oil sands extraction"]
for i in list_name:
 df_test = df_rev_2014_21[df_rev_2014_21["NAICSTitleEn"]== i].copy()

In [ ]:
df_test.shape

(5884, 18)

In [ ]:
target_df.isna().sum()

In [ ]:
t_minus_18_values = []  # List to store t_minus_18 values

for index, row in df_test.iterrows():
    curr_year = row['NPRI_Report_ReportYear']
    facil_name = row['FacilityName']
    cas_name = row['Cas']
    found = False  # Flag to check if a matching row is found

    for index, row_1 in df_test.iterrows():
        if row_1['NPRI_Report_ReportYear'] == curr_year - 1 and row_1['FacilityName'] == facil_name and row_1['Cas'] == cas_name:
            t_minus_18_values.append(row_1["Quantity"])
            found = True
            break

    if not found:
        t_minus_18_values.append(None)  # If no matching row is found, append None

df_test["t_minus1"] = t_minus_18_values

In [ ]:
t_minus_18_values = []  # List to store t_minus_18 values

for index, row in df_test.iterrows():
    curr_year = row['NPRI_Report_ReportYear']
    facil_name = row['FacilityName']
    cas_name = row['Cas']
    found = False  # Flag to check if a matching row is found

    for index, row_1 in df_test.iterrows():
        if row_1['NPRI_Report_ReportYear'] == curr_year - 2 and row_1['FacilityName'] == facil_name and row_1['Cas'] == cas_name:
            t_minus_18_values.append(row_1["Quantity"])
            found = True
            break

    if not found:
        t_minus_18_values.append(None)  # If no matching row is found, append None

df_test["t_minus2"] = t_minus_18_values

In [ ]:
df_test.to_csv("df_test_year_minus.csv")

In [ ]:
df_rev_2014_21 = df_rev_2014_21.rename(columns={"t_minus1": "YearMinus1", "t_minus2": "YearMinus2"})


In [ ]:
df_test.dropna(inplace = True)


In [ ]:
df_test = pd.read_csv("df_test_year_minus.csv")

In [ ]:
df_test.isna().sum()

In [ ]:
df_test.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
df_test.dropna(subset=["FacilityName","Latitude","Longitude"], inplace=True)

In [ ]:
df_test.drop(columns=['Substance comments for disposals and all media for 2022',"Substance comments by category for 2022"], inplace=True)

In [ ]:
df_test.fillna(method='ffill', inplace=True)
df_test.fillna(method='bfill', inplace=True)

In [ ]:
df_test.isna().sum()

**Encoding the dataset**

In [ ]:
df_test.head()

#Target_df

In [ ]:
target_df_1.drop(columns=['Substance comments for disposals and all media for 2022',"Substance comments by category for 2022"], inplace=True)

In [ ]:
target_df_1.head()

In [ ]:
list_catg = ["NAICSTitleEn","ProvinceEn","Category","Group","Detail_NPRI_SubstanceInfo_NameEn","FacilityName","Company name","Cas",'NAICSPrimary', 'NpriID', 'Detail_UnitType_NameEn']

In [ ]:
target_df_1.head()

,YearMinus1,YearMinus2,NPRI_Report_ReportYear,Quantity,Latitude,Longitude,Number of employees,NAICSTitleEn_mean_encoded,ProvinceEn_mean_encoded,Category_mean_encoded,Group_mean_encoded,Detail_NPRI_SubstanceInfo_NameEn_mean_encoded,FacilityName_mean_encoded,Company name_mean_encoded,Cas_mean_encoded,NAICSPrimary_mean_encoded,NpriID_mean_encoded,Detail_UnitType_NameEn_mean_encoded
5985,117.940284,96.850435,2022,257170.654,57.0033,-111.4661,6274.0,235932.048014,235932.048014,742885.054548,691163.992494,40699.924975,192008.539307,192008.539307,40699.924975,235932.048014,192008.539307,395743.653802
5986,117.940284,96.850435,2022,109801.371,57.0033,-111.4661,6274.0,235932.048014,235932.048014,266151.731125,345489.539347,40699.924975,192008.539307,192008.539307,40699.924975,235932.048014,192008.539307,395743.653802
5987,117.940284,96.850435,2022,29738.098,57.0033,-111.4661,6274.0,235932.048014,235932.048014,116463.068298,345489.539347,40699.924975,192008.539307,192008.539307,40699.924975,235932.048014,192008.539307,395743.653802
5988,117.940284,96.850435,2022,6502.445,57.0033,-111.4661,6274.0,235932.048014,235932.048014,62308.263975,345489.539347,40699.924975,192008.539307,192008.539307,40699.924975,235932.048014,192008.539307,395743.653802
5989,117.940284,96.850435,2022,121.162,57.0033,-111.4661,6274.0,235932.048014,235932.048014,122.119792,98.166477,40699.924975,192008.539307,192008.539307,40699.924975,235932.048014,192008.539307,395743.653802


In [ ]:
for catg in list_catg:  # Encoding usign mean encoding method
    try:
        mean_target = target_df_1.groupby(catg)['Quantity'].mean()
        target_df_1[catg + '_mean_encoded'] = target_df_1[catg].map(mean_target)
    except KeyError as e:


        print(f"KeyError: {e} not found in DataFrame.")

In [ ]:
target_df_1.drop(columns=['NAICSTitleEn','ProvinceEn','Category','Group','Detail_NPRI_SubstanceInfo_NameEn','FacilityName','Company name','Cas','NAICSPrimary', 'NpriID', 'Detail_UnitType_NameEn'], inplace=True)

In [ ]:
target_df_1.to_csv("target_df_1.csv")

**Normalization**

In [ ]:
# Applying normalization on this featurs.
list_col= ['YearMinus1', 'YearMinus2', 'Quantity', 'Number of employees',
       'NAICSTitleEn_mean_encoded', 'ProvinceEn_mean_encoded',
       'Group_mean_encoded', 'Company name_mean_encoded',
       'NAICSPrimary_mean_encoded',
       'Substance comments for disposals and all media for 2022_mean_encoded',
       'Substance comments by category for 2022_mean_encoded']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
target_df[list_col] = scaler.fit_transform(target_df[list_col])

# df_rev_2014_21

**Encoding**

In [ ]:
for catg in list_catg:
    try:
        mean_target = df_test.groupby(catg)['Quantity'].mean()
        df_test[catg + '_mean_encoded'] = df_test[catg].map(mean_target)
    except KeyError as e:
        print(f"KeyError: {e} not found in DataFrame.")

In [ ]:
df_test.drop(columns=['NAICSTitleEn','ProvinceEn','Category','Group','Detail_NPRI_SubstanceInfo_NameEn','FacilityName','Company name','Cas','NAICSPrimary', 'NpriID', 'Detail_UnitType_NameEn'], inplace=True)

In [ ]:
df_test.to_csv("df_test_missing-value_removed.csv")


**Normalization**

In [ ]:
# Applying normalization on this featurs.
list_col= ['t_minus2', 't_minus1', 'Quantity', 'Number of employees',
       'NAICSTitleEn_mean_encoded', 'ProvinceEn_mean_encoded',
       'Group_mean_encoded', 'Company name_mean_encoded',
       'NAICSPrimary_mean_encoded',
       'Substance comments for disposals and all media for 2022_mean_encoded',
       'Substance comments by category for 2022_mean_encoded']

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_test[list_col] = scaler.fit_transform(df_test[list_col])

**Feature Selection based on Pearson value and correlation_matrix ML dataset**

In [ ]:

correlation_matrix = target_df.corr().abs()


target_corr = correlation_matrix['Quantity']


selected_features = target_corr[target_corr > 0.2].index.tolist()


selected_features.remove('Quantity')


print("Selected features based on Pearson correlation coefficient:")
print(selected_features)

Selected features based on Pearson correlation coefficient:
['FacilityName_mean_encoded', 'NpriID_mean_encoded']


In [ ]:
X_with_const = sm.add_constant(target_df.drop(columns=['Quantity']))


model = sm.OLS(target_df['Quantity'], X_with_const).fit()
print(model.summary())

p_values = model.pvalues
selected_features = target_df.columns[1:][p_values < 0.05]

print("Selected Features based on p-value less than 0.05:")
print(selected_features)


                            OLS Regression Results                            
Dep. Variable:               Quantity   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                  0.071
Method:                 Least Squares   F-statistic:                     146.3
Date:                Mon, 08 Apr 2024   Prob (F-statistic):               0.00
Time:                        20:07:45   Log-Likelihood:            -7.7411e+05
No. Observations:               30325   AIC:                         1.548e+06
Df Residuals:                   30308   BIC:                         1.548e+06
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                                                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------

In [ ]:
target_df.drop(columns=['Category_mean_encoded',
       'Detail_NPRI_SubstanceInfo_NameEn_mean_encoded', 'Cas_mean_encoded',
       'Detail_UnitType_NameEn_mean_encoded','FacilityName_mean_encoded', 'NpriID_mean_encoded'], inplace=True)

df_rev_2014_21.drop(columns=['Category_mean_encoded',
       'Detail_NPRI_SubstanceInfo_NameEn_mean_encoded', 'Cas_mean_encoded',
       'Detail_UnitType_NameEn_mean_encoded','FacilityName_mean_encoded', 'NpriID_mean_encoded'], inplace=True)

`